In [2]:
import pandas as pd
df_final_demo = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_demo.txt")
df_final_experiment_clients = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_experiment_clients.txt")
df_final_web_data_pt_1 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_1.txt")
df_final_web_data_pt_2 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_2.txt")


In [3]:
# Concating df_final_web p1 and p2 and merging them with df_final_experiment_clients
df_final_web_data = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)
df_web_experiment = pd.merge(
    df_final_web_data,
    df_final_experiment_clients,
    on="client_id",
    how="left"
)

In [4]:
# Saving them as csv
#  df_web_experiment.to_csv("df_web_experiment.csv")

----------------------

In [5]:
# SettingWithCopyWarning fixing
df_web_experiment = df_web_experiment.copy()

# Making num_visit col (client_id)
client_visits = (
    df_web_experiment
    .groupby('client_id')['visit_id']
    .nunique()
    .reset_index(name='num_visits')
)

df_web_experiment.loc[:, 'num_visits'] = (
    df_web_experiment
    .groupby('client_id')['visit_id']
    .transform('nunique')
)

# Making num_steps (visit_id)
df_web_experiment.loc[:, 'num_steps'] = (
    df_web_experiment
    .groupby('visit_id')['process_step']
    .transform('count')
)

# Fixing date_time col 
df_web_experiment.loc[:, 'date_time'] = pd.to_datetime(df_web_experiment['date_time'], errors='coerce')

# Making last_step col 
df_web_experiment.loc[:, 'last_step'] = (
    df_web_experiment
    .sort_values(['visit_id', 'date_time'])
    .groupby('visit_id')['process_step']
    .transform('last')
)

# Making visit duration col (minutes:seconds format)
df_web_experiment.loc[:, 'visit_duration'] = (
    df_web_experiment
    .groupby('visit_id')['date_time']
    .transform(
        lambda x: str(
            f"{int((x.max() - x.min()).total_seconds() // 60):02d}:{int((x.max() - x.min()).total_seconds() % 60):02d}"
        ) if pd.notnull(x.max()) and pd.notnull(x.min()) else None
    )
).astype('string')

# filling missing in variation to unknown 
df_web_experiment['Variation'] = df_web_experiment['Variation'].fillna('unknown')

----------------

In [6]:
# df with the unique visit id with Variation
df_unique_visits = df_web_experiment.drop_duplicates(subset='visit_id', keep='first')


In [7]:
df_unique_visits

,client_id,visitor_id,visit_id,process_step,date_time,Variation,num_visits,num_steps,last_step,visit_duration
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,2,8,step_3,10:45
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,1,5,confirm,01:59
13,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,1,15,confirm,25:03
24,1982004,618934751_10082078917,132494692_73738638511_70823,confirm,2017-04-17 12:02:42,Test,1,5,confirm,03:00
29,9294070,489274391_63794562906,116640698_27328181095_188045,start,2017-04-17 11:16:28,Control,1,2,start,00:15
...,...,...,...,...,...,...,...,...,...,...
755375,2443347,818414719_69544570654,148166449_67114410166_524360,confirm,2017-05-30 19:18:18,unknown,3,1,confirm,00:00
755376,9895983,473024645_56027518531,498981662_93503779869_272484,confirm,2017-06-15 19:55:09,unknown,1,6,confirm,11:04
755382,9563467,358754208_29929976792,172141804_88150434297_748843,step_1,2017-05-18 08:38:37,unknown,2,2,step_1,01:08
755384,9668240,388766751_9038881013,922267647_3096648104_968866,confirm,2017-05-24 19:07:35,unknown,1,20,confirm,23:01


In [8]:
df_final_demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0
